## Système de recommandation de jeux de données

In [2]:
## Installing libraries
#!pip install numpy pandas scikit-learn nltk pickle5 regex

In [70]:
# Importing useful libraries

import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from datetime import datetime
import pickle5 as pickle


In [2]:
# files importation
canada_metadata=pd.read_csv('metadata/canada_metadata.csv')
france_metadata=pd.read_csv('metadata/france_metadata.csv',sep=";")

In [ ]:
# metadata exploration

In [3]:
canada_metadata.head()

,ref_number,title_en,title_fr,description_en,description_fr,publisher_en,publisher_fr,date_published,language,size,eligible_for_release,program_alignment_architecture_en,program_alignment_architecture_fr,date_released,portal_url_en,portal_url_fr,owner_org,owner_org_title
0,OD-2017-00001,Proactive Disclosure of Travel Expenses for Se...,Divulgation proactive des frais de voyage des ...,This data provides information on the travel e...,Ce données fournissent des renseignements sur ...,ATSSC,SCDATA,2014-11-01,en+fr,NaN,Y,1.1 Program: Tribunal Specialized and Expert S...,Services de soutien spécialisé et expert aux t...,NaN,NaN,NaN,atssc-scdata,Administrative Tribunals Support Service of Ca...
1,OD-2017-00002,Proactive Disclosure of Hospitality Expenses f...,Divulgation proactive des frais d'accueil des ...,This data provides information on the hospital...,Ce données fournissent des renseignements sur ...,ATSSC,SCDATA,2014-11-01,en+fr,NaN,Y,1.1 Program: Tribunal Specialized and Expert S...,Services de soutien spécialisé et expert aux t...,NaN,NaN,NaN,atssc-scdata,Administrative Tribunals Support Service of Ca...
2,OD-2017-00003,"Proactive Disclosure of Contracts over $10,000",Divulgation proactive des marchés de plus de 1...,This data provides information on contracts is...,Ces données fournissent des renseignements sur...,ATSSC,SCDATA,2014-11-01,en+fr,NaN,Y,1.1 Program: Tribunal Specialized and Expert S...,Services de soutien spécialisé et expert aux t...,NaN,NaN,NaN,atssc-scdata,Administrative Tribunals Support Service of Ca...
3,OD-2017-00004,Disclosure of Position Reclassifications,Divulgation des reclassifications de postes,This data provides information on the reclassi...,Ces données fournissent des renseignements su...,ATSSC,SCDATA,2014-11-01,en+fr,NaN,Y,1.1 Program: Tribunal Specialized and Expert S...,Services de soutien spécialisé et expert aux t...,NaN,NaN,NaN,atssc-scdata,Administrative Tribunals Support Service of Ca...
4,OD-2017-00005,Completed Access to Information Requests,Demandes d’accès à l’information complétées,This data provides information on completed ac...,Ces données fournissent des renseignements su...,ATSSC,SCDATA,2014-11-01,en+fr,NaN,Y,1.1 Program: Tribunal Specialized and Expert S...,Services de soutien spécialisé et expert aux t...,NaN,NaN,NaN,atssc-scdata,Administrative Tribunals Support Service of Ca...


In [4]:
canada_metadata.describe()

,size
count,9.411000e+03
mean,7.441605e+12
std,5.356111e+14
min,0.000000e+00
25%,1.050000e+05
50%,3.000000e+09
75%,3.000000e+09
max,5.000051e+16


In [5]:
france_metadata.head()

,id,title,slug,url,organization,organization_id,description,frequency,license,temporal_coverage.start,...,private,featured,created_at,last_modified,tags,metric.discussions,metric.issues,metric.reuses,metric.followers,metric.views
0,613fe932c8633c842fb25368,Illustrations et photos - Jardins Ephémères - ...,illustrations-et-photos-jardins-ephemeres-orle...,http://www.data.gouv.fr/fr/datasets/illustrati...,Orléans Métropole,57ed2fe9c751df236179df72,Illustrations pour l'opération Jardins Ephémères,unknown,Licence Ouverte / Open Licence version 2.0,NaN,...,False,False,2021-09-14T02:13:38.775000,2021-09-10T15:05:59,environnement,0,0,0,0,0
1,613fe932531a0701082bd965,Jardins éphémères - Orléans Métropole,jardins-ephemeres-orleans-metropole,http://www.data.gouv.fr/fr/datasets/jardins-ep...,Orléans Métropole,57ed2fe9c751df236179df72,Liste des jardins éphémères. Edition 2021. \n\...,unknown,Licence Ouverte / Open Licence version 2.0,NaN,...,False,False,2021-09-14T02:13:38.494000,2021-09-13T21:01:24,environnement,0,0,0,0,0
2,613fe932d1e20a30342bd965,Plantations - Jardins Ephémères - Orléans Métr...,plantations-jardins-ephemeres-orleans-metropole,http://www.data.gouv.fr/fr/datasets/plantation...,Orléans Métropole,57ed2fe9c751df236179df72,Inventaires des plantations réalisées dans le ...,unknown,Licence Ouverte / Open Licence version 2.0,NaN,...,False,False,2021-09-14T02:13:38.161000,2021-09-13T21:30:07,environnement,0,0,0,0,0
3,613fe92a1b97b27f262bd965,Producteurs locaux d'espèces végétales - Orléa...,producteurs-locaux-despeces-vegetales-orleans-...,http://www.data.gouv.fr/fr/datasets/producteur...,Orléans Métropole,57ed2fe9c751df236179df72,Producteurs locaux d'espèces végétales,unknown,Licence Ouverte / Open Licence version 2.0,NaN,...,False,False,2021-09-14T02:13:30.876000,2021-09-13T21:00:03,environnement,0,0,0,0,0
4,613fe92adefb3cc9512bd965,Production d'espèces végétales - Jardins Ephém...,production-despeces-vegetales-jardins-ephemere...,http://www.data.gouv.fr/fr/datasets/production...,Orléans Métropole,57ed2fe9c751df236179df72,Productions réalisées par les producteurs loca...,unknown,Licence Ouverte / Open Licence version 2.0,NaN,...,False,False,2021-09-14T02:13:30.869000,2021-09-13T21:30:05,environnement,0,0,0,0,0


In [6]:
france_metadata.describe()

,metric.discussions,metric.issues,metric.reuses,metric.followers,metric.views
count,50608.000000,50608.000000,50608.000000,50608.000000,50608.000000
mean,0.150786,0.009366,0.086429,0.116424,84.244309
std,3.013136,0.162087,1.128809,1.443997,1781.156908
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000
max,358.000000,20.000000,146.000000,134.000000,167441.000000


In [7]:
canada_metadata.columns

Index(['ref_number', 'title_en', 'title_fr', 'description_en',
       'description_fr', 'publisher_en', 'publisher_fr', 'date_published',
       'language', 'size', 'eligible_for_release',
       'program_alignment_architecture_en',
       'program_alignment_architecture_fr', 'date_released', 'portal_url_en',
       'portal_url_fr', 'owner_org', 'owner_org_title'],
      dtype='object')

In [8]:
canada_metadata[['date_published','date_released','portal_url_fr','program_alignment_architecture_fr']].iloc[1:15,:]

,date_published,date_released,portal_url_fr,program_alignment_architecture_fr
1,2014-11-01,NaN,NaN,Services de soutien spécialisé et expert aux t...
2,2014-11-01,NaN,NaN,Services de soutien spécialisé et expert aux t...
3,2014-11-01,NaN,NaN,Services de soutien spécialisé et expert aux t...
4,2014-11-01,NaN,NaN,Services de soutien spécialisé et expert aux t...
5,2018-08-09,2018-08-10,https://ouvert.canada.ca/data/fr/dataset/6adae...,"Accès aux marchés, négociations, compétitivité..."
6,2018-11-23,2018-11-23,https://ouvert.canada.ca/data/fr/dataset/b22cd...,"Sciences, innovation, adoption et durabilité"
7,2018-12-14,2018-12-14,https://ouvert.canada.ca/data/fr/dataset/97add...,"Sciences, innovation, adoption et durabilité"
8,2018-12-14,2018-12-14,https://ouvert.canada.ca/data/fr/dataset/97add...,"Sciences, innovation, adoption et durabilité"
9,2018-12-14,2018-12-14,https://ouvert.canada.ca/data/fr/dataset/97add...,"Sciences, innovation, adoption et durabilité"
10,2009-04-01,2016-09-24,http://ouvert.canada.ca/data/fr/dataset/16d2f8...,"Sciences, innovation, adoption et durabilité"


In [9]:
france_metadata.columns

Index(['id', 'title', 'slug', 'url', 'organization', 'organization_id',
       'description', 'frequency', 'license', 'temporal_coverage.start',
       'temporal_coverage.end', 'spatial.granularity', 'spatial.zones',
       'private', 'featured', 'created_at', 'last_modified', 'tags',
       'metric.discussions', 'metric.issues', 'metric.reuses',
       'metric.followers', 'metric.views'],
      dtype='object')

In [10]:
## We will merge the tables together to have an unique table
## What do we need for our final table?

# title, description, url, organization,published date, topic


In [11]:
# The wanted columns
columns=['id','title','description','url','organization',
                               'published_date','topic']

In [12]:
# The columns equivalent for each dataset
canada_col = ['ref_number','title_fr','description_fr','portal_url_fr',
              'owner_org_title','date_published','program_alignment_architecture_fr']
france_col= ['id','title','description','url','organization','last_modified','tags']

In [13]:
# data normalization
def norm_data(data,col):
    new_data=data.copy()
    new_data = new_data[col]  # selecting the wanted columns
    
    new_data.rename(columns=dict(zip(col,columns)),inplace=True)
    new_data.dropna(inplace=True)                                  # dropping Nan values
    new_data['published_date']=pd.to_datetime(new_data['published_date'], errors='coerce')  # formatting the date
    #new_data.dropna(subset=['published_date'],inplace=True)
    return new_data
    

In [14]:
# Getting the normalized datasets 
canada_metadata=norm_data(canada_metadata,canada_col)
france_metadata=norm_data(france_metadata,france_col)

In [15]:
# metadata fusion
metadata=pd.concat([canada_metadata,france_metadata])


In [16]:
metadata

,id,title,description,url,organization,published_date,topic
5,ODI-2018-01035,Compte de productivité pour l’agriculture cana...,Le Compte de productivité pour l’agriculture c...,https://ouvert.canada.ca/data/fr/dataset/6adae...,Agriculture and Agri-Food Canada | Agriculture...,2018-08-09,"Accès aux marchés, négociations, compétitivité..."
6,ODI-2018-01037,Chimie de l'eau de Swift Current - Étude de tr...,La quantité et la qualité de l'eau actuelle de...,https://ouvert.canada.ca/data/fr/dataset/b22cd...,Agriculture and Agri-Food Canada | Agriculture...,2018-11-23,"Sciences, innovation, adoption et durabilité"
7,ODI-2018-01022,Programme de pesticides à usage limité État d'...,Le Programme des pesticides à usage limité du ...,https://ouvert.canada.ca/data/fr/dataset/97add...,Agriculture and Agri-Food Canada | Agriculture...,2018-12-14,"Sciences, innovation, adoption et durabilité"
8,ODI-2018-01023,État des présentations de programmes de pestic...,Le Programme des pesticides à usage limité du ...,https://ouvert.canada.ca/data/fr/dataset/97add...,Agriculture and Agri-Food Canada | Agriculture...,2018-12-14,"Sciences, innovation, adoption et durabilité"
9,ODI-2018-01024,Statistiques d'essais du Programme de pesticid...,Le tableau suivant présente de l'information c...,https://ouvert.canada.ca/data/fr/dataset/97add...,Agriculture and Agri-Food Canada | Agriculture...,2018-12-14,"Sciences, innovation, adoption et durabilité"
...,...,...,...,...,...,...,...
50603,5e9d5c68a82c577228180956,Réseau et résultats Nitrates des eaux souterra...,Identification du réseau Nitrates des eaux sou...,http://www.data.gouv.fr/fr/datasets/reseau-et-...,DREAL Bourgogne-Franche-Comté,2020-11-12,"donnees-vecteur,dreal-bfc,eau,hydrographie,mas..."
50604,5e9d5c68b0818e7a9318094a,Les arbres remarquables de Bourgogne: arbres u...,Localisation des arbres uniques de l'inventair...,http://www.data.gouv.fr/fr/datasets/les-arbres...,DREAL Bourgogne-Franche-Comté,2021-02-02,"arbre,classe-des-dicotyledones,dispositif-pere..."
50605,5ecd3633169cfc9e3353b35a,Base Adresse Locale de Grand Besançon Métropole,Adresses au format Base Adresse Locale (BAL) c...,http://www.data.gouv.fr/fr/datasets/base-adres...,GRAND BESANCON METROPOLE,2021-02-02,"adresses,bal,base-adresse-locale,grand-besanco..."
50606,5da117b29ce2e747fb520796,Plan d'Occupation des Sols de la commune des S...,Zonages du Plan d'Occupation des Sols de la co...,http://www.data.gouv.fr/fr/datasets/fr-2413004...,Arles Crau Camargue Montagnette,1998-03-13,"document-durbanisme,geographie,land-use,plan-d..."


In [17]:
# Creating a column that will be used to search for information
metadata['search_string'] = metadata.topic.str.lower() + " " + metadata.title.str.lower() + " " + metadata.topic.str.lower()

In [18]:
metadata

,id,title,description,url,organization,published_date,topic,search_string
5,ODI-2018-01035,Compte de productivité pour l’agriculture cana...,Le Compte de productivité pour l’agriculture c...,https://ouvert.canada.ca/data/fr/dataset/6adae...,Agriculture and Agri-Food Canada | Agriculture...,2018-08-09,"Accès aux marchés, négociations, compétitivité...","accès aux marchés, négociations, compétitivité..."
6,ODI-2018-01037,Chimie de l'eau de Swift Current - Étude de tr...,La quantité et la qualité de l'eau actuelle de...,https://ouvert.canada.ca/data/fr/dataset/b22cd...,Agriculture and Agri-Food Canada | Agriculture...,2018-11-23,"Sciences, innovation, adoption et durabilité","sciences, innovation, adoption et durabilité c..."
7,ODI-2018-01022,Programme de pesticides à usage limité État d'...,Le Programme des pesticides à usage limité du ...,https://ouvert.canada.ca/data/fr/dataset/97add...,Agriculture and Agri-Food Canada | Agriculture...,2018-12-14,"Sciences, innovation, adoption et durabilité","sciences, innovation, adoption et durabilité p..."
8,ODI-2018-01023,État des présentations de programmes de pestic...,Le Programme des pesticides à usage limité du ...,https://ouvert.canada.ca/data/fr/dataset/97add...,Agriculture and Agri-Food Canada | Agriculture...,2018-12-14,"Sciences, innovation, adoption et durabilité","sciences, innovation, adoption et durabilité é..."
9,ODI-2018-01024,Statistiques d'essais du Programme de pesticid...,Le tableau suivant présente de l'information c...,https://ouvert.canada.ca/data/fr/dataset/97add...,Agriculture and Agri-Food Canada | Agriculture...,2018-12-14,"Sciences, innovation, adoption et durabilité","sciences, innovation, adoption et durabilité s..."
...,...,...,...,...,...,...,...,...
50603,5e9d5c68a82c577228180956,Réseau et résultats Nitrates des eaux souterra...,Identification du réseau Nitrates des eaux sou...,http://www.data.gouv.fr/fr/datasets/reseau-et-...,DREAL Bourgogne-Franche-Comté,2020-11-12,"donnees-vecteur,dreal-bfc,eau,hydrographie,mas...","donnees-vecteur,dreal-bfc,eau,hydrographie,mas..."
50604,5e9d5c68b0818e7a9318094a,Les arbres remarquables de Bourgogne: arbres u...,Localisation des arbres uniques de l'inventair...,http://www.data.gouv.fr/fr/datasets/les-arbres...,DREAL Bourgogne-Franche-Comté,2021-02-02,"arbre,classe-des-dicotyledones,dispositif-pere...","arbre,classe-des-dicotyledones,dispositif-pere..."
50605,5ecd3633169cfc9e3353b35a,Base Adresse Locale de Grand Besançon Métropole,Adresses au format Base Adresse Locale (BAL) c...,http://www.data.gouv.fr/fr/datasets/base-adres...,GRAND BESANCON METROPOLE,2021-02-02,"adresses,bal,base-adresse-locale,grand-besanco...","adresses,bal,base-adresse-locale,grand-besanco..."
50606,5da117b29ce2e747fb520796,Plan d'Occupation des Sols de la commune des S...,Zonages du Plan d'Occupation des Sols de la co...,http://www.data.gouv.fr/fr/datasets/fr-2413004...,Arles Crau Camargue Montagnette,1998-03-13,"document-durbanisme,geographie,land-use,plan-d...","document-durbanisme,geographie,land-use,plan-d..."


In [ ]:
# Preprocessing the strings by removing commas, common words in french
def string_preprocess(s):
    
    s = re.sub(r'[^\w\s]',' ',s)
    s.lower()
    
    return " ".join(set(s.split()) - set(stopwords.words('french')))

In [22]:
# testing the preprocessing function
query='covid est le nouveau virus'
print(string_preprocess(query))

nouveau covid virus


In [20]:
metadata['search_string']=metadata['search_string'].map(string_preprocess)

In [21]:
# reindexing by simply using the order in the table
metadata.reset_index(drop=True,inplace=True)

In [23]:
docs = list(metadata['search_string'])

In [25]:
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(docs)


In [27]:
# Saving X and the Vectorizer
with open("X.pkl", 'wb') as file:  
    pickle.dump(X, file)
with open("vectorizer.pkl",'wb') as file : 
    pickle.dump(vectorizer,file)

In [27]:
# clustering the documents 
model = KMeans(100)
model.fit(X_csr) # should be X

KMeans(n_clusters=100)

In [32]:
# saving the cluster model
#with open("cluster_model.pkl", 'wb') as file:  
#    pickle.dump(model, file)

In [29]:
## loading the model
with open("cluster_model.pkl", 'rb') as file:  
    model = pickle.load(file)
 

In [30]:
# adding the cluster labels to each row in the dataset
metadata['cluster'] = model.labels_

In [31]:
# saving the final dataset
with open("metadata.pkl",'wb') as file : 
    pickle.dump(metadata,file)

In [ ]:
# Loading all the required files and models
with open('cluster_model.pkl', 'rb') as file:  
    model = pickle.load(file)

with open('X.pkl', 'rb') as file:  
    X = pickle.load(file)
    
with open('metadata.pkl','rb') as file:
    metadata=pickle.load(file)
    
with open('vectorizer.pkl','rb') as file:
    vectorizer = pickle.load(file)

In [68]:
# The function which returns the result for a query
def result(query,metadata=metadata,X=X,vectorizer=vectorizer,model=model, start_date=datetime(1800,1,1,0,0,0), end_date=datetime.now(),limit=100):
    # preprocessing the query
    query = string_preprocess(query)
    query_vec=vectorizer.transform([query])  # query vector
    
    # filtering with date
    L=list(metadata[ (metadata['cluster']==int(model.predict(query_vec)) ) & ((start_date<=metadata.published_date) & (metadata.published_date <= end_date)) ].index)
    G=[0 for g in range(len(L))]
    #print( cosine_similarity(q_csr,q_csr )[0][0] )
    for indx, i in enumerate(L):
        # Computing the similarity with each document
        m= cosine_similarity(query_vec,X[i,:].reshape(1,-1)) [0][0]
        #m=np.dot(query_vec.T,X[:,i].reshape(1,-1))
        #print(m)
        G[indx]=np.array([i,m])
    # Sorting the results
    M=np.array(sorted(G, key=lambda x: x[1],reverse=True))
    
    return list(metadata.iloc[M[:,0][:limit],3])
    
    

In [69]:
# Testing on a specific query
result(query='agriculture',limit=100)

['http://www.data.gouv.fr/fr/datasets/agriculture/',
 'http://www.data.gouv.fr/fr/datasets/prix-des-terres-agricoles-nd/',
 'http://www.data.gouv.fr/fr/datasets/prix-des-terres-agricoles/',
 'http://www.data.gouv.fr/fr/datasets/icpe-agricoles-en-pays-de-la-loire/',
 'http://www.data.gouv.fr/fr/datasets/agreste-cartographie-des-donnees-du-recensement-agricole/',
 'http://www.data.gouv.fr/fr/datasets/zones-agricoles-defavorisees-en-bourgogne/',
 'http://www.data.gouv.fr/fr/datasets/test-emprise-3/',
 'http://www.data.gouv.fr/fr/datasets/zones-agricoles-defavorisees-en-correze/',
 'http://www.data.gouv.fr/fr/datasets/zones-agricoles-defavorisees-en-alsace/',
 'http://www.data.gouv.fr/fr/datasets/zones-agricoles-defavorisees-sur-le-departement-du-cher/',
 'http://www.data.gouv.fr/fr/datasets/zones-agricoles-defavorisees-dans-la-vienne/',
 'http://www.data.gouv.fr/fr/datasets/zones-agricoles-defavorisees-dans-la-vienne-1/',
 'http://www.data.gouv.fr/fr/datasets/maet-nombre-de-contrats/',
 '

In [ ]:
# another test
result(query='covid 19',limit=100)